<a href="https://colab.research.google.com/github/olivia-sp/smart-sv-dev/blob/main/ssv_getRoadPOI_v0_15_olivia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
#도로정보DB - https://data.gg.go.kr/portal/data/dataset/searchDatasetPage.do
#GPS 데이터 기반 연산 로직 개발
#ver0.1 - 현재 위치 반경 계산 및 성남시 분당구 도로정보 필터링_신호등위치, 횡단보도위치, 도로안전표지위치
#ver0.15 - 성남시 분당구 일반음식점 추가

In [55]:
import pandas as pd
import os
import numpy as np

#DB(csv)파일읽어오기

##신호등위치(성남시분당구)

In [56]:
import os

if os.path.isfile('/content/drive/MyDrive/Colab Notebooks/data_set/신호등현황(개방표준).csv') == True:
  print('already')
else:
  #os.mkdir('data_set')
  %cd data_set
  !wget https://raw.githubusercontent.com/olivia-sp/smart-sv-dev/main/data_set/신호등현황(개방표준).csv

% cd /content


already
/content


In [57]:
CURR_PATH = os.getcwd()

os.listdir(CURR_PATH)
#DATA_PATH = os.path.join(CURR_PATH,'data_set')
RAW_PATH = CURR_PATH + '/drive/MyDrive/Colab Notebooks/data_set/신호등현황(개방표준).csv'



In [58]:
raw_data_traffLgt = pd.read_csv(RAW_PATH, index_col=0, encoding='cp949')
raw_data_traffLgt.info()
raw_data_traffLgt.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2972 entries, 9 to 9
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   시군구명           2972 non-null   object 
 1   도로종류           2972 non-null   object 
 2   도로노선번호         1430 non-null   object 
 3   도로노선명          2972 non-null   object 
 4   도로노선방향         2972 non-null   int64  
 5   소재지도로명주소       564 non-null    object 
 6   소재지지번주소        1420 non-null   object 
 7   위도             1420 non-null   float64
 8   경도             1420 non-null   float64
 9   신호기설치방식        2972 non-null   int64  
 10  도로형태           2972 non-null   int64  
 11  주도로여부          2972 non-null   object 
 12  신호등관리번호        2972 non-null   object 
 13  신호등구분          2972 non-null   int64  
 14  신호등색종류         2972 non-null   int64  
 15  신호등화방식         2972 non-null   int64  
 16  신호등화순서         2972 non-null   object 
 17  신호등화시간         2972 non-null   object 
 18  광원종류       

시군구명                0
도로종류                0
도로노선번호           1542
도로노선명               0
도로노선방향              0
소재지도로명주소         2408
소재지지번주소          1552
위도               1552
경도               1552
신호기설치방식             0
도로형태                0
주도로여부               0
신호등관리번호             0
신호등구분               0
신호등색종류              0
신호등화방식              0
신호등화순서              0
신호등화시간              0
광원종류                0
신호제어방식              0
신호시간결정방식            0
점멸등운영여부             0
점멸등운영시작시각           0
점멸등운영종료시각           0
보행자작동신호기유무          0
잔여시간표시기유무           0
시각장애인용음향신호기유무       0
도로안내표지일련번호       2972
관리기관명               0
관리기관전화번호            0
데이터기준일자             0
dtype: int64

In [59]:
if 'index' not in raw_data_traffLgt.columns:
  raw_data_traffLgt.reset_index(inplace=True)

In [63]:
raw_data_traffLgt['소재지도로명주소'] = raw_data_traffLgt['소재지도로명주소'].astype('str')
filter_city = raw_data_traffLgt['소재지도로명주소'].str.contains('분당구')
df_traffLgt_SNcity = raw_data_traffLgt[filter_city]

In [64]:
df_traffLgt_SNcity.reset_index(inplace=True, drop=True)

In [65]:
df_traffLgt_SNcity.head(2)

,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,소재지지번주소,위도,경도,...,점멸등운영여부,점멸등운영시작시각,점멸등운영종료시각,보행자작동신호기유무,잔여시간표시기유무,시각장애인용음향신호기유무,도로안내표지일련번호,관리기관명,관리기관전화번호,데이터기준일자
0,9,성남시분당구,시도,판교원로 82번길,판교원로 82번길,3,경기도 성남시 분당구 판교원로82번길 60,경기도 성남시 분당구 운중동 916,37.384058,127.072437,...,Y,00:00,05:30,N,N,N,NaN,성남시청,031-729-3673,2018-08-31
1,9,성남시분당구,시도,판교역로,판교역로,3,경기도 성남시 분당구 판교역로 188,경기도 성남시 분당구 삼평동 642,37.397392,127.109988,...,N,00:00,00:00,N,Y,N,NaN,성남시청,031-729-3673,2018-08-31


##횡단보도위치(성남시분당구)

In [66]:
import os

if os.path.isfile('/content/drive/MyDrive/Colab Notebooks/data_set/횡단보도현황(개방표준).csv') == True:
  print('already')
else:
  #os.mkdir('data_set')
  %cd data_set
  !wget https://raw.githubusercontent.com/olivia-sp/smart-sv-dev/main/data_set/횡단보도현황(개방표준).csv

% cd /content


already
/content


In [67]:
CURR_PATH = os.getcwd()

os.listdir(CURR_PATH)
#DATA_PATH = os.path.join(CURR_PATH,'data_set')
RAW_PATH = CURR_PATH + '/drive/MyDrive/Colab Notebooks/data_set/횡단보도현황(개방표준).csv'



In [68]:
raw_data_crossWlk = pd.read_csv(RAW_PATH, index_col=0, encoding='cp949')
raw_data_crossWlk.info()
raw_data_crossWlk.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5003 entries, 9 to 9
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   시군구명        5003 non-null   object 
 1   도로명         5003 non-null   object 
 2   소재지도로명주소    575 non-null    object 
 3   소재지지번주소     2710 non-null   object 
 4   횡단보도관리번호    5003 non-null   object 
 5   횡단보도종류      5003 non-null   int64  
 6   자전거횡단도겸용여부  670 non-null    object 
 7   고원식적용여부     5003 non-null   object 
 8   위도          2710 non-null   float64
 9   경도          2710 non-null   float64
 10  차로수         5003 non-null   int64  
 11  횡단보도폭       5003 non-null   float64
 12  횡단보도연장      5003 non-null   float64
 13  보행자신호등유무    5003 non-null   object 
 14  보행자작동신호기유무  907 non-null    object 
 15  음향신호기설치여부   907 non-null    object 
 16  녹색신호시간      104 non-null    float64
 17  적색신호시간      0 non-null      float64
 18  교통섬유무       5003 non-null   object 
 19  보도턱낮춤여부     1818 non-null   ob

시군구명             0
도로명              0
소재지도로명주소      4428
소재지지번주소       2293
횡단보도관리번호         0
횡단보도종류           0
자전거횡단도겸용여부    4333
고원식적용여부          0
위도            2293
경도            2293
차로수              0
횡단보도폭            0
횡단보도연장           0
보행자신호등유무         0
보행자작동신호기유무    4096
음향신호기설치여부     4096
녹색신호시간        4899
적색신호시간        5003
교통섬유무            0
보도턱낮춤여부       3185
점자블록유무           0
집중조명시설유무      3955
관리기관명            0
관리기관전화번호         0
데이터기준일자          0
dtype: int64

In [69]:
if 'index' not in raw_data_crossWlk.columns:
  raw_data_crossWlk.reset_index(inplace=True)

In [70]:
raw_data_crossWlk['소재지도로명주소'] = raw_data_crossWlk['소재지도로명주소'].astype('str')
filter_city = raw_data_crossWlk['소재지도로명주소'].str.contains('분당구')
df_crossWlk_SNcity = raw_data_crossWlk[filter_city]

In [71]:
df_crossWlk_SNcity.reset_index(inplace=True, drop=True)

In [72]:
df_crossWlk_SNcity.head(2)

,시도명,시군구명,도로명,소재지도로명주소,소재지지번주소,횡단보도관리번호,횡단보도종류,자전거횡단도겸용여부,고원식적용여부,위도,...,음향신호기설치여부,녹색신호시간,적색신호시간,교통섬유무,보도턱낮춤여부,점자블록유무,집중조명시설유무,관리기관명,관리기관전화번호,데이터기준일자
0,9,성남시분당구,운중로 243번길,경기도 성남시 분당구 운중로243번길 12,경기도 성남시 분당구 판교동 612-4번지,-,4,NaN,N,37.390521,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30
1,9,성남시분당구,동판교로,"경기도 성남시 분당구 동판교로 212 (삼평동, 봇들마을6단지아파트)",경기도 성남시 분당구 삼평동 728번지 봇들마을6단지아파트,-,4,NaN,N,37.401425,...,NaN,NaN,NaN,N,Y,Y,Y,성남시청,031-729-3672,2020-06-30


##도로안전표지위치(성남시분당구)

In [73]:
import os

if os.path.isfile('/content/drive/MyDrive/Colab Notebooks/data_set/도로안전표지현황(개방표준).csv') == True:
  print('already')
else:
  #os.mkdir('data_set')
  %cd data_set
  !wget https://raw.githubusercontent.com/olivia-sp/smart-sv-dev/main/data_set/도로안전표지현황(개방표준).csv

% cd /content


already
/content


In [74]:
CURR_PATH = os.getcwd()

os.listdir(CURR_PATH)
#DATA_PATH = os.path.join(CURR_PATH,'data_set')
RAW_PATH = CURR_PATH + '/drive/MyDrive/Colab Notebooks/data_set/도로안전표지현황(개방표준).csv'



In [75]:
raw_data_safeSgn = pd.read_csv(RAW_PATH, index_col=0, encoding='cp949')
raw_data_safeSgn.info()
raw_data_safeSgn.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1140 entries, nan to 107.0
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   도로종류        1140 non-null   object 
 1   도로노선번호      1140 non-null   object 
 2   도로노선명       1140 non-null   object 
 3   도로노선방향      1140 non-null   int64  
 4   도로형태        651 non-null    float64
 5   차로수         0 non-null      float64
 6   도로폭         0 non-null      float64
 7   소재지도로명주소    599 non-null    object 
 8   소재지지번주소     1140 non-null   object 
 9   위도          1140 non-null   float64
 10  경도          1140 non-null   float64
 11  안전표지구분      1140 non-null   int64  
 12  안전표지종별일련번호  1140 non-null   object 
 13  주행제한속도      12 non-null     float64
 14  안전표지설명      1140 non-null   object 
 15  지주형식        256 non-null    float64
 16  제2외국어표기여부   0 non-null      float64
 17  설치일자        0 non-null      float64
 18  관리기관명       1140 non-null   object 
 19  관리기관전화번호    1140 non-n

도로종류             0
도로노선번호           0
도로노선명            0
도로노선방향           0
도로형태           489
차로수           1140
도로폭           1140
소재지도로명주소       541
소재지지번주소          0
위도               0
경도               0
안전표지구분           0
안전표지종별일련번호       0
주행제한속도        1128
안전표지설명           0
지주형식           884
제2외국어표기여부     1140
설치일자          1140
관리기관명            0
관리기관전화번호         0
데이터기준일자          0
dtype: int64

In [76]:
if 'index' not in raw_data_safeSgn.columns:
  raw_data_safeSgn.reset_index(inplace=True)

In [77]:
raw_data_safeSgn['소재지도로명주소'] = raw_data_safeSgn['소재지도로명주소'].astype('str')
filter_city = raw_data_safeSgn['소재지도로명주소'].str.contains('분당구')
df_safeSgn_SNcity = raw_data_safeSgn[filter_city]

In [78]:
df_safeSgn_SNcity.reset_index(inplace=True, drop=True)

##일반음식점위치(성남시분당구)

In [117]:
import os

if os.path.isfile('/content/drive/MyDrive/Colab Notebooks/data_set/일반음식점(복어취급)현황.csv') == True:
  print('already')
else:
  #os.mkdir('data_set')
  %cd data_set
  !wget https://raw.githubusercontent.com/olivia-sp/smart-sv-dev/main/data_set/일반음식점(복어취급)현황.csv

% cd /content


already
/content


In [118]:
CURR_PATH = os.getcwd()

os.listdir(CURR_PATH)
#DATA_PATH = os.path.join(CURR_PATH,'data_set')
RAW_PATH = CURR_PATH + '/drive/MyDrive/Colab Notebooks/data_set/일반음식점(복어취급)현황.csv'



In [119]:
raw_data_dining = pd.read_csv(RAW_PATH, index_col=0, encoding='cp949')
raw_data_dining.info()
raw_data_dining.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 72322 entries, 가평군 to nan
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   사업장명      72322 non-null  object 
 1   인허가일자     72322 non-null  int64  
 2   영업상태명     72322 non-null  object 
 3   폐업일자      14600 non-null  float64
 4   년도        180 non-null    float64
 5   다중이용업소여부  0 non-null      float64
 6   총시설규모(㎡)  0 non-null      float64
 7   위생업종명     180 non-null    object 
 8   위생업태명     72267 non-null  object 
 9   소재지도로명주소  71852 non-null  object 
 10  소재지지번주소   72319 non-null  object 
 11  소재지우편번호   71898 non-null  float64
 12  WGS84위도   71888 non-null  float64
 13  WGS84경도   71888 non-null  float64
dtypes: float64(7), int64(1), object(6)
memory usage: 8.3+ MB


사업장명            0
인허가일자           0
영업상태명           0
폐업일자        57722
년도          72142
다중이용업소여부    72322
총시설규모(㎡)    72322
위생업종명       72142
위생업태명          55
소재지도로명주소      470
소재지지번주소         3
소재지우편번호       424
WGS84위도       434
WGS84경도       434
dtype: int64

In [133]:
if 'index' not in raw_data_dining.columns:
  raw_data_dining.reset_index(inplace=True)

In [146]:
raw_data_dining['사업장명'] = raw_data_dining['사업장명'].astype('str')
raw_data_dining['소재지도로명주소'] = raw_data_dining['소재지도로명주소'].astype('str')
filter_city = raw_data_dining['소재지도로명주소'].str.contains('분당구')
df_dining_SNcity0 = raw_data_dining[filter_city]

In [147]:
df_dining_SNcity0['영업상태명'] = df_dining_SNcity0['영업상태명'].astype('str')
filter_city = df_dining_SNcity0['영업상태명'].str.contains('영업|영업중|운영중')
df_dining_SNcity = df_dining_SNcity0[filter_city]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [148]:
df_dining_SNcity.reset_index(inplace=True, drop=True)

In [149]:
df_dining_SNcity

,index,시군명,사업장명,인허가일자,영업상태명,폐업일자,년도,다중이용업소여부,총시설규모(㎡),위생업종명,위생업태명,소재지도로명주소,소재지지번주소,소재지우편번호,WGS84위도,WGS84경도
0,20954,성남시,포키오그릴,20010602,영업,NaN,NaN,NaN,NaN,NaN,식육(숯불구이),"경기도 성남시 분당구 매화로37번길 7-12 (야탑동,1층)",경기도 성남시 분당구 야탑동 388-4 1층,13505.0,37.410972,127.131957
1,20955,성남시,그린포유,20220510,영업,NaN,NaN,NaN,NaN,NaN,기타 휴게음식점,"경기도 성남시 분당구 황새울로319번길 6, 텍스타워 지하1층 B107호 (서현동)",경기도 성남시 분당구 서현동 272-3 텍스타워 지하1층 B107호,13590.0,37.385473,127.120797
2,20958,성남시,스타벅스 네이버1784점,20220407,영업,NaN,NaN,NaN,NaN,NaN,커피숍,"경기도 성남시 분당구 정자일로 95, 네이버 1784 3층 일부호 (정자동)","경기도 성남시 분당구 정자동 178-4 네이버 1784, 3층 일부호",13561.0,37.358859,127.105201
3,20959,성남시,콤마PC방,20220406,영업,NaN,NaN,NaN,NaN,NaN,기타 휴게음식점,"경기도 성남시 분당구 돌마로 85, 지하1층 비03호의 일부호 (금곡동)",경기도 성남시 분당구 금곡동 152 지하1층 비03호의 일부호,13618.0,37.350388,127.111342
4,20962,성남시,(주)보나비아티제 분당서현점,20220426,영업,NaN,NaN,NaN,NaN,NaN,기타,"경기도 성남시 분당구 분당로 55, 분당 퍼스트타워 1층 일부, 2층 일부호 (서현동)","경기도 성남시 분당구 서현동 266-1 분당 퍼스트타워 1층 일부, 2층 일부호",13591.0,37.383015,127.121498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1591,24135,성남시,바다복집,19930419,운영중,NaN,1993.0,NaN,NaN,일반음식점,복어취급,"경기도 성남시 분당구 내정로165번길 38 (수내동,금호상가2층224호)",경기도 성남시 분당구 수내동 32번지 금호상가2층224호,13597.0,37.375979,127.117117
1592,24136,성남시,미락복집,20021102,운영중,NaN,2002.0,NaN,NaN,일반음식점,복어취급,"경기도 성남시 분당구 구미로9번길 25 (구미동, 대맥주차빌딩 1층108호)",경기도 성남시 분당구 구미동 182번지 대맥주차빌딩 1층108호,13637.0,37.340378,127.109937
1593,24137,성남시,진미복집,19940525,운영중,NaN,1994.0,NaN,NaN,일반음식점,복어취급,경기도 성남시 분당구 효자길 21 (서현동),경기도 성남시 분당구 서현동 327-8번지,13578.0,37.378200,127.138216
1594,24138,성남시,현복집(판교점),20160624,운영중,NaN,2016.0,NaN,NaN,일반음식점,복어취급,"경기도 성남시 분당구 대왕판교로606번길 10 (백현동, 알파리움 타워1동 2층 2...","경기도 성남시 분당구 백현동 531번지 알파리움 타워1동 2층 215호, 216호",13530.0,37.395832,127.107583


#fake GPS 더미

In [46]:
!pip3 install Faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 16.2 MB/s 


In [47]:
from faker import Faker
fake = Faker('ko_KR')
# for _ in range(5):
#   latlng = fake.local_latlng('KR')
#   print(latlng)

latlng = fake.local_latlng('KR')
print(latlng)
lat = float (latlng[0])
lon = float (latlng[1])
my_latlng = (lat, lon)
print(my_latlng)

('37.1759', '128.9889', 'T‚Äôaebaek', 'KR', 'Asia/Seoul')
(37.1759, 128.9889)


# 현재 위치 반경 계산

##도로정보

In [79]:
!pip3 install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [49]:
### 공식문서 예시
# from haversine import haversine, Unit

# lyon = (45.7597, 4.8422) # (lat, lon)
# paris = (48.8567, 2.3508)

# haversine(lyon, paris)
# >> 392.2172595594006  # in kilometers

# haversine(lyon, paris, unit=Unit.MILES)
# >> 243.71201856934454  # in miles

# # you can also use the string abbreviation for units:
# haversine(lyon, paris, unit='mi')
# >> 243.71201856934454  # in miles

# haversine(lyon, paris, unit=Unit.NAUTICAL_MILES)
# >> 211.78037755311516  # in nautical miles

In [80]:
df_traffLgt_SNcity.info()
df_crossWlk_SNcity.info()
df_safeSgn_SNcity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 32 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   시도명            298 non-null    int64  
 1   시군구명           298 non-null    object 
 2   도로종류           298 non-null    object 
 3   도로노선번호         298 non-null    object 
 4   도로노선명          298 non-null    object 
 5   도로노선방향         298 non-null    int64  
 6   소재지도로명주소       298 non-null    object 
 7   소재지지번주소        298 non-null    object 
 8   위도             298 non-null    float64
 9   경도             298 non-null    float64
 10  신호기설치방식        298 non-null    int64  
 11  도로형태           298 non-null    int64  
 12  주도로여부          298 non-null    object 
 13  신호등관리번호        298 non-null    object 
 14  신호등구분          298 non-null    int64  
 15  신호등색종류         298 non-null    int64  
 16  신호등화방식         298 non-null    int64  
 17  신호등화순서         298 non-null    object 
 18  신호등화시간    

In [81]:
#판교만도중앙연구소
my_latlng = (37.404704734328, 127.10535530866)

In [103]:
from haversine import haversine, Unit
distance_compare_list_traffLgt = []
distance_compare_list_crosWlk = []
distance_compare_list_safeSgn = []

df_new_traffLgt = pd.DataFrame(columns = df_traffLgt_SNcity.columns)
df_new_crosWlk = pd.DataFrame(columns = df_crossWlk_SNcity.columns)
df_new_safeSgn = pd.DataFrame(columns = df_safeSgn_SNcity.columns)

## traffic light
for i in range(df_traffLgt_SNcity.shape[0]):
    db_traffLgt = (df_traffLgt_SNcity["위도"][i], df_traffLgt_SNcity["경도"][i])
    traffLgt_num = df_traffLgt_SNcity["신호등관리번호"][i]
    result = haversine(db_traffLgt, my_latlng, unit='m')
    #반경 500m 이내
    if result <= 500:
      distance_compare_list_traffLgt.append(result)
      print('trafficLight', db_traffLgt)
      filter_lat=df_traffLgt_SNcity['위도'] == db_traffLgt[0]
      filter_lon=df_traffLgt_SNcity['경도'] == db_traffLgt[1]
      filter_num=df_traffLgt_SNcity["신호등관리번호"] == traffLgt_num #같은 위치 신호등 구분
      df = df_traffLgt_SNcity[filter_lat & filter_lon & filter_num]
      df_new_traffLgt = pd.concat([df_new_traffLgt, df], axis = 0)
df_new_traffLgt['radius'] = distance_compare_list_traffLgt
df_new_traffLgt.reset_index(inplace=True, drop=True)

# cross walk
for i in range(df_crossWlk_SNcity.shape[0]):
    db_crosWlk = (df_crossWlk_SNcity["위도"][i], df_crossWlk_SNcity["경도"][i])
    result = haversine(db_crosWlk, my_latlng, unit='m')
    #반경 500m 이내
    if result <= 500:
      distance_compare_list_crosWlk.append(result)
      print('crossWalk', db_crosWlk)
      filter_lat=df_crossWlk_SNcity['위도'] == db_crosWlk[0]
      filter_lon=df_crossWlk_SNcity['경도'] == db_crosWlk[1]
      df = df_crossWlk_SNcity[filter_lat & filter_lon]
      df_new_crosWlk = pd.concat([df_new_crosWlk, df], axis = 0)
df_new_crosWlk['radius'] = distance_compare_list_crosWlk
df_new_crosWlk.reset_index(inplace=True, drop=True)

# safe sign
for i in range(df_safeSgn_SNcity.shape[0]):
    db_safesgn = (df_safeSgn_SNcity["위도"][i], df_safeSgn_SNcity["경도"][i])
    safesgn_num = df_safeSgn_SNcity["안전표지종별일련번호"][i]
    result = haversine(db_safesgn, my_latlng, unit='m')
    #반경 500m 이내
    if result <= 500:
      distance_compare_list_safeSgn.append(result)
      print('safeSign', db_safesgn)
      filter_lat=df_safeSgn_SNcity['위도'] == db_safesgn[0]
      filter_lon=df_safeSgn_SNcity['경도'] == db_safesgn[1]
      filter_num=df_safeSgn_SNcity["안전표지종별일련번호"] == safesgn_num #같은 위치 안전표지 구분
      df = df_safeSgn_SNcity[filter_lat & filter_lon & filter_num]
      df_new_safeSgn = pd.concat([df_new_safeSgn, df], axis = 0)
df_new_safeSgn['radius'] = distance_compare_list_safeSgn
df_new_safeSgn.reset_index(inplace=True, drop=True)

trafficLight (37.4037259458, 127.0998775385)
trafficLight (37.4045092638, 127.1005562139)
trafficLight (37.4041629952, 127.1051525762)
trafficLight (37.4044202324, 127.1097384586)
trafficLight (37.4033105416, 127.1010177224)
trafficLight (37.4033105416, 127.1010177224)
trafficLight (37.4019750211, 127.1034329076)
trafficLight (37.4045044843, 127.1061808333)
trafficLight (37.400611472, 127.1069321778)
trafficLight (37.4033935026, 127.1092739361)
trafficLight (37.4027630214, 127.1090245731)
crossWalk (37.405076, 127.100406)
crossWalk (37.403169, 127.102582)
crossWalk (37.402375, 127.109682)
crossWalk (37.403942, 127.109649)
crossWalk (37.402307, 127.102577)
crossWalk (37.404494, 127.101492)
crossWalk (37.400852, 127.102599)
crossWalk (37.403902, 127.106456)
crossWalk (37.403183, 127.105057)
crossWalk (37.403178, 127.109643)
safeSign (37.4033935026, 127.1092739361)
safeSign (37.4033105416, 127.1010177224)
safeSign (37.4019750211, 127.1034329076)
safeSign (37.4037259458, 127.0998775385)
sa

In [112]:
df_new_traffLgt.columns

Index(['시도명', '시군구명', '도로종류', '도로노선번호', '도로노선명', '도로노선방향', '소재지도로명주소',
       '소재지지번주소', '위도', '경도', '신호기설치방식', '도로형태', '주도로여부', '신호등관리번호', '신호등구분',
       '신호등색종류', '신호등화방식', '신호등화순서', '신호등화시간', '광원종류', '신호제어방식', '신호시간결정방식',
       '점멸등운영여부', '점멸등운영시작시각', '점멸등운영종료시각', '보행자작동신호기유무', '잔여시간표시기유무',
       '시각장애인용음향신호기유무', '도로안내표지일련번호', '관리기관명', '관리기관전화번호', '데이터기준일자',
       'radius'],
      dtype='object')

In [111]:
df_new_crosWlk.columns

Index(['시도명', '시군구명', '도로명', '소재지도로명주소', '소재지지번주소', '횡단보도관리번호', '횡단보도종류',
       '자전거횡단도겸용여부', '고원식적용여부', '위도', '경도', '차로수', '횡단보도폭', '횡단보도연장',
       '보행자신호등유무', '보행자작동신호기유무', '음향신호기설치여부', '녹색신호시간', '적색신호시간', '교통섬유무',
       '보도턱낮춤여부', '점자블록유무', '집중조명시설유무', '관리기관명', '관리기관전화번호', '데이터기준일자',
       'radius'],
      dtype='object')

In [110]:
df_new_safeSgn.columns

Index(['안전표지일련번호', '도로종류', '도로노선번호', '도로노선명', '도로노선방향', '도로형태', '차로수', '도로폭',
       '소재지도로명주소', '소재지지번주소', '위도', '경도', '안전표지구분', '안전표지종별일련번호', '주행제한속도',
       '안전표지설명', '지주형식', '제2외국어표기여부', '설치일자', '관리기관명', '관리기관전화번호', '데이터기준일자',
       'radius'],
      dtype='object')

## 일반음식점

In [125]:
!pip3 install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [150]:
df_dining_SNcity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596 entries, 0 to 1595
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     1596 non-null   int64  
 1   시군명       1596 non-null   object 
 2   사업장명      1596 non-null   object 
 3   인허가일자     1596 non-null   int64  
 4   영업상태명     1596 non-null   object 
 5   폐업일자      0 non-null      float64
 6   년도        5 non-null      float64
 7   다중이용업소여부  0 non-null      float64
 8   총시설규모(㎡)  0 non-null      float64
 9   위생업종명     5 non-null      object 
 10  위생업태명     1596 non-null   object 
 11  소재지도로명주소  1596 non-null   object 
 12  소재지지번주소   1596 non-null   object 
 13  소재지우편번호   1574 non-null   float64
 14  WGS84위도   1574 non-null   float64
 15  WGS84경도   1574 non-null   float64
dtypes: float64(7), int64(2), object(7)
memory usage: 199.6+ KB


In [167]:
#판교만도중앙연구소
my_latlng = (37.404704734328, 127.10535530866)

In [168]:
from haversine import haversine, Unit
distance_compare_list_dining = []

df_new_dining = pd.DataFrame(columns = df_dining_SNcity.columns)

## traffic light
for i in range(df_dining_SNcity.shape[0]):
    db_dining = (df_dining_SNcity["WGS84위도"][i], df_dining_SNcity["WGS84경도"][i])
    dining_name = df_dining_SNcity["사업장명"][i]
    result = haversine(db_dining, my_latlng, unit='m')
    #반경 500m 이내
    if result <= 300:
      distance_compare_list_dining.append(result)
      #print('dining', db_dining)
      filter_lat=df_dining_SNcity['WGS84위도'] == db_dining[0]
      filter_lon=df_dining_SNcity['WGS84경도'] == db_dining[1]
      filter_name=df_dining_SNcity["사업장명"] == dining_name #같은 위치 식당 구분
      df = df_dining_SNcity[filter_lat & filter_lon & filter_name]
      df_new_dining = pd.concat([df_new_dining, df], axis = 0)
      #print(df)
df_new_dining['radius'] = distance_compare_list_dining
df_new_dining.reset_index(inplace=True, drop=True)


In [162]:
df_new_dining

,index,시군명,사업장명,인허가일자,영업상태명,폐업일자,년도,다중이용업소여부,총시설규모(㎡),위생업종명,위생업태명,소재지도로명주소,소재지지번주소,소재지우편번호,WGS84위도,WGS84경도,radius
0,21069,성남시,해킹,20220506,영업,NaN,NaN,NaN,NaN,NaN,기타,"경기도 성남시 분당구 대왕판교로 670, 유스페이스2 2층 208호 (삼평동)","경기도 성남시 분당구 삼평동 682 유스페이스2, 2층 208호",13494.0,37.402065,127.106059,300.082676
1,21638,성남시,미닛트,20170515,영업,NaN,NaN,NaN,NaN,NaN,기타,"경기도 성남시 분당구 대왕판교로 670 (삼평동, 유스페이스2. 1층 112호)",경기도 성남시 분당구 삼평동 682 유스페이스2. 1층 112호,13494.0,37.402065,127.106059,300.082676
2,21675,성남시,제이콥스,20201112,영업,NaN,NaN,NaN,NaN,NaN,커피숍,"경기도 성남시 분당구 대왕판교로 670, 유스페이스2 지하1층 B107호 (삼평동)",경기도 성남시 분당구 삼평동 682 유스페이스2 지하1층 B107호,13494.0,37.402065,127.106059,300.082676
3,21720,성남시,아트비어컴퍼니,20130510,영업,NaN,NaN,NaN,NaN,NaN,경양식,"경기도 성남시 분당구 대왕판교로 670 (삼평동, 유스페이스2. 지하1층 B108호)",경기도 성남시 분당구 삼평동 682 유스페이스2. 지하1층 B108호,13494.0,37.402065,127.106059,300.082676
4,21935,성남시,와이키키(Waikiki),20180627,영업,NaN,NaN,NaN,NaN,NaN,경양식,"경기도 성남시 분당구 판교로319번길 13, 디테라스 1층 107호 (삼평동)",경기도 성남시 분당구 삼평동 691 디테라스 1층 107호,13488.0,37.403643,127.109124,353.181580
5,22015,성남시,커피온리 판교점,20120416,영업,NaN,NaN,NaN,NaN,NaN,커피숍,"경기도 성남시 분당구 대왕판교로 670 (삼평동, 유스페이스2 지상1층 131호)",경기도 성남시 분당구 삼평동 682 유스페이스2 지상1층 131호,13494.0,37.402065,127.106059,300.082676
6,22127,성남시,커피클라우드나인,20210518,영업,NaN,NaN,NaN,NaN,NaN,커피숍,"경기도 성남시 분당구 대왕판교로 670, 유스페이스2 1층 103호 (삼평동)",경기도 성남시 분당구 삼평동 682 유스페이스2 1층 103호,13494.0,37.402065,127.106059,300.082676
7,22274,성남시,됐소,20140516,영업,NaN,NaN,NaN,NaN,NaN,한식,"경기도 성남시 분당구 대왕판교로 670 (삼평동, 유스페이스2 지하1층 B101호)",경기도 성남시 분당구 삼평동 682 유스페이스2 지하1층 B101호,13494.0,37.402065,127.106059,300.082676
8,22358,성남시,슈퍼타이 판교테크노밸리점,20160628,영업,NaN,NaN,NaN,NaN,NaN,기타,"경기도 성남시 분당구 판교로319번길 14, 1층 2호 (삼평동, 성남판교 경기행복주택)",경기도 성남시 분당구 삼평동 700 성남판교 경기행복주택 1층 2호,13488.0,37.404372,127.108770,303.853531
9,22569,성남시,씨유 판교현무점,20220203,영업,NaN,NaN,NaN,NaN,NaN,편의점,"경기도 성남시 분당구 판교로319번길 14, 근린생활시설동 1층 5호의 일부호 (삼...",경기도 성남시 분당구 삼평동 700 성남판교 경기행복주택 근린생활시설 1층 5호의 ...,13488.0,37.404372,127.108770,303.853531
